This is submission code for the Kaggle competition

Playground Series - Season 3, Episode 5

https://www.kaggle.com/competitions/playground-series-s3e5

which came in position 153 of 903, which is okay but worse than the previous competition result, despite this notebook being more sophisticated.

# Packages and Data

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
from collections import defaultdict
import random

# Scikit stuff
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.pipeline import *
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.neighbors import *
from sklearn.utils import resample

# XGB
from xgboost import XGBRegressor, XGBClassifier

# TPOT
from tpot import TPOTRegressor, TPOTClassifier

In [2]:
data = pd.read_csv('C:/Users/medion/playground-series-s3e5/train.csv')
eval_data = pd.read_csv('C:/Users/medion/playground-series-s3e5/test.csv')
sample_submit = pd.read_csv('C:/Users/medion/playground-series-s3e5/sample_submission.csv')

data = data.drop("Id", axis=1)
eval_data = eval_data.drop('Id', axis=1)

target_col = 'quality'

# Data Processing

We have classes for the train and test data. It contains possible functions we can use for data processing. Some are used for data exploration so are not used in the final notebook.

In [3]:
class DataProcess():
    def __init__(self, data):
        self.data_original = data
        self.data = self.data_original.copy()
        
    def histplot(self):
        '''
        Plots a histogram for each feature, this might be called after transforming features,
        use original_data_histplot() to call this with the original data
        '''
        for column in self.data.columns:
            sns.histplot(data = self.data[column])
            plt.show()
            
    def original_data_histplot(self):
        '''
        Plots a histogram for each feature in the original data
        '''
        for column in self.data_original.columns:
            sns.histplot(data = self.data_original[column])
            plt.show()
    
    def drop(self, col_list):
        '''
        Drops a list of features
        '''
        for col in col_list:
            self.data = self.data.drop(col, axis=1)
    
    def value_counts(self):
        '''
        Returns data counts for each feature
        '''
        for column in self.data.columns:
            print(f"{column}")
            print(self.data[column].value_counts())
            print("-------------------------------------")
        
    def remove_outliers(self):
        '''
        Removes outliers from the dataset. These were chosen by inspection. Here we have certain target values
        only appearing a few times, so we should be careful not to remove data with one of those target values.
        '''
        self.data = self.data[self.data['residual sugar'] < 10]
        self.data = self.data[self.data['chlorides'] < 0.3]
        self.data = self.data[self.data['chlorides'] > 0.02]
        self.data = self.data[self.data['free sulfur dioxide'] < 60]
        self.data = self.data[self.data['total sulfur dioxide'] < 160]
        self.data = self.data[self.data['sulphates'] < 1.9]
        self.data = self.data[self.data['pH'] > 2.8]
        self.data = self.data[self.data['volatile acidity'] < 1.5]
        self.data = self.data[self.data['residual sugar'] > 1.3]
        
        self.data = self.data.reset_index()
        self.data = self.data.drop('index', axis = 1)

    def show_correlation(self):
        '''
        Show n highest and lowest correlated variables for each variable
        '''
        corr = pd.DataFrame(self.data.corr())
        for column in self.data.columns:
            print(f'Three lowest correlation for {column}')
            print(pd.DataFrame(corr[column]).sort_values(column).iloc[0:12])
            print('----------------------------------------------------')
            print(f'Three highest correlation for {column}')
            print(pd.DataFrame(corr[column]).sort_values(column).iloc[-12-1:-1])
            print('-----------------------------------------------------')
            
    def power_transform(self):
        target_list = [target_col]
        power_data = [item for item in self.data.columns if item not in target_list]
        
        for item in power_data:
            pt = PowerTransformer()
            self.data[item+' *'] = pt.fit_transform(pd.DataFrame(self.data[item]))
            self.data = self.data.drop(item, axis=1)

    def add_features(self): 
        '''
        Add features to the data set, this was done by adding using the below functions to add all
        products, ratios, reciprocals, roots, logs, and exps and seeing which were useful. We also drop pH,
        which gave better results.
        '''
        self.data['alcohol * pH'] = self.data['alcohol'] * self.data['pH']
        self.data['density / sulphates'] = self.data['density'] / self.data['sulphates']
        self.data['sulphates / total sulfur dioxide'] = self.data['sulphates'] / self.data['total sulfur dioxide']
        self.data['density / chlorides'] = self.data['density'] / self.data['chlorides']
        self.data['total sulfur dioxide / density'] = self.data['total sulfur dioxide'] / self.data['density']
        self.data['citric acid * pH'] = self.data['citric acid'] * self.data['pH']
        self.data['total sulfur dioxide / volatile acidity'] = self.data['total sulfur dioxide'] / self.data['volatile acidity']
        self.data['density / fixed acidity'] = self.data['density'] / self.data['fixed acidity']
        self.data['sulphates * fixed acidity'] = self.data['sulphates'] * self.data['fixed acidity']
        self.data['density * chlorides'] = self.data['density'] * self.data['chlorides']
        self.data['1/free sulfur dioxide'] = 1/self.data['free sulfur dioxide']
        self.data['log sulphates'] = np.log(self.data['sulphates'])
        self.data['root sulphates'] = np.sqrt(self.data['sulphates'])
        self.data['log pH'] = np.log(self.data['pH'])
        self.data['root pH'] = np.sqrt(self.data['pH'])
        self.data['exp pH'] = np.exp(self.data['pH'])
        self.data['1/alcohol'] = 1/self.data['alcohol']
        self.data['1/density'] = 1/self.data['density']
        self.data['1/chlorides'] = 1/self.data['chlorides']
        self.data['1/residual sugar'] = 1/self.data['residual sugar']
        self.data['1/pH'] = 1/self.data['pH']
        self.data['1/fixed acidity'] = 1/self.data['fixed acidity']
        self.data['1/total sulfur dioxide'] = 1/self.data['total sulfur dioxide'] 
        
        self.data = self.data.drop('pH', axis=1)
        
    def add_all_products(self):
        '''
        Adds features corresponding to the products of each pair of features
        '''
        pairs = []
        names = []
        for column in self.data_original.columns:
            for column_pair in self.data_original.columns:
                if column != target_col and column_pair != target_col:
                    pairs.append(self.data[column] * self.data[column_pair])
                    names.append(f'{column} * {column_pair}')
        new = pd.DataFrame(pairs).transpose()
        new.columns = names
        self.data = pd.concat([self.data, new], axis=1)
        
    def add_all_ratios(self):
        '''
        Adds features corresponding to the ratios of each pair of features (provided we don't divide by 0)
        '''
        pairs = []
        names = []
        for column in self.data_original.columns:
            for column_pair in self.data_original.columns:
                if column_pair != column and column != target_col and column_pair != target_col \
                and (self.data[column] / self.data[column_pair]).sum() != np.inf:
                    pairs.append(self.data[column] / self.data[column_pair])
                    names.append(f'{column} / {column_pair}')
        new = pd.DataFrame(pairs).transpose()
        new.columns = names
        self.data = pd.concat([self.data, new], axis=1)
        
    def add_all_reciprocals(self):
        '''
        Adds features corresponding to the reciprocals of each feature (provided we don't divide by 0)
        '''
        reciprocal_cols = []
        names = []
        for column in self.data_original.columns:
            if column != target_col and (1/self.data[column]).sum() != np.inf:
                reciprocal_cols.append(1/self.data[column])
                names.append(f'1/{column}')
        new = pd.DataFrame(reciprocal_cols).transpose()
        new.columns = names
        self.data = pd.concat([self.data, new], axis=1)
        
    def add_all_roots(self):
        '''
        Adds features corresponding to the square roots of each feature
        '''
        for column in self.data_original.columns:
            if column != target_col and (self.data[column] >= 0).all():
                self.data[f'root {column}'] = np.sqrt(self.data[column])
                
    def add_all_logs(self):
        '''
        Adds features corresponding to the log of each feature (provided features are >0)
        '''
        for column in self.data_original.columns:
            if column != target_col and (self.data[column] > 0).all():
                self.data[f'log {column}'] = np.log(self.data[column])

    def add_all_exp(self):
        '''
        Adds features corresponding to the exp of each feature
        '''
        for column in self.data_original.columns:
            if column != target_col:
                self.data[f'exp {column}'] = np.exp(self.data[column])
        
    def general_add_all_features(self):
        '''
        Adds all features from above, allowing us to see which are useful.
        '''
        self.add_all_roots()
        self.add_all_logs()
        self.add_all_exp()
        self.add_all_reciprocals()
        self.add_all_products()
        self.add_all_ratios()
        return
        
    def pipeline(self):
        '''
        Pipeline of chosen data processing methods
        '''
        self.remove_outliers()
        self.add_features()
        self.power_transform()  
        
# Process data and split features, target     

train_class = DataProcess(data)
train_class.pipeline()
X_train_origin = train_class.data.drop(target_col, axis=1)
y_train = train_class.data[target_col]

# We need to label encode since y_train values are 3,4,5,6,7,8, which XGBoost doesn't like
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train = pd.DataFrame(y_train)
y_train.columns = [target_col]

In [4]:
class EvalProcess(DataProcess):
    def __init__(self, data):
        self.data_original = data
        self.data = self.data_original.copy()
    
    def cols(self):
        '''
        Run this to ensure train and eval data have the same columns
        '''
        eval_cols = [
            column for column in self.data.columns if column in train_class.data.columns
        ]
        self.data = self.data[eval_cols]
    
    def pipeline(self):
        self.add_features()
        self.power_transform()
        self.cols()

# Process eval data        
        
eval_class = EvalProcess(eval_data)
eval_class.pipeline()

eval_data_origin = eval_class.data

# Model Selection

We search over a selection of parameters to find the best XGB model. This is a much tidier version of the code from the previous playground series competition.

Since we have a number of target values that only appear a few times, we use skikit's resample to randomly choose the number of target values that appear. We also randomly choose a selection of feature columns.

This has a feature that, if we see a poor kappa result on one of the K-fold slices then we will just skip to the next iteration, saving time.

In [5]:
# A selection of parameters, these will be randomly selected from.

n_estimators_values = [50, 75, 100, 125, 150, 200, 250, 300, 350]
learning_rate_values = np.linspace(0.02, 0.16, 10)
max_depth_values = [2,3,4,5]
subsample_values = [0.25, 0.50, 0.75, 0.90]
colsample_bytree_values = [0.25, 0.50, 0.75, 0.90]

# Some target values appear a lot (max_value_count), some very few times (min_value_count) and some in the 
# middle (mid value count). We need these values for resampling.

max_value_count = y_train.value_counts().max()
mid_value_count = y_train.value_counts()[4]
min_value_count = y_train.value_counts().min()

# We select at random the number of target values in include

class_values_small = np.linspace(10, 75, 10).round().astype('int')
class_values_mid = np.linspace(mid_value_count-100, mid_value_count+100, 20).round().astype('int')
class_values_large = np.linspace(300, max_value_count, 10).round().astype('int')

# Fixed parameters

cv_folds = 5
tuning_iterations = 1000
random.seed(2201020)
skf_seed = random.randint(0, 2023)

# K-fold split

skf = StratifiedKFold(n_splits = cv_folds, random_state = skf_seed, shuffle = True)

# Initialize for results storage

tuning_results = defaultdict(list)
valid_predictions = []
eval_predictions = []
feature_importances = []
cols_selection = []
completed_iterations = tuning_iterations

# The loop

for step in range(tuning_iterations):
    # Choose random XGB parameters

    n_estimators = random.choice(n_estimators_values)
    learning_rate = random.choice(learning_rate_values)
    max_depth = random.choice(max_depth_values)
    subsample = random.choice(subsample_values)
    colsample_bytree = random.choice(colsample_bytree_values)
    
    # Adding extra copies of data with particular target values, since these are under-represented
    
    class_0_value = random.choice(class_values_small)
    class_1_value = random.choice(class_values_small)
    class_2_value = random.choice(class_values_large)
    class_3_value = random.choice(class_values_large)
    class_4_value = random.choice(class_values_mid)
    class_5_value = random.choice(class_values_small)
    
    X_train = X_train_origin
    eval_data = eval_data_origin
    
    # Random column selection, here selecting at least one third
    num_cols = random.randrange(X_train_origin.shape[1]//3, X_train_origin.shape[1])
    rand_cols = random.sample(X_train_origin.columns.to_list(), num_cols)
    cols_selection.append(rand_cols)
    
    X_train = X_train[rand_cols]
    
    # Initlialize for results storage
    
    kappas = []
    test_probs = []
    iter_valid_predictions = []
    iter_eval_predictions = []
    iter_feature_importance = np.zeros(num_cols)

    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        X_train_split, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_split, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # We need to join X and y before we resample
        
        merged = X_train_split.join(y_train_split)
        
        classes = []
        for j in range(6):
            this_class = merged[merged[target_col] == j]
            classes.append(this_class)
        
        # Resample classes based on random selection above
        
        classes[0] = resample(classes[0], n_samples = class_0_value)
        classes[1] = resample(classes[1], n_samples = class_1_value)
        classes[2] = resample(classes[2], n_samples = class_2_value)
        classes[3] = resample(classes[3], n_samples = class_3_value)
        classes[4] = resample(classes[4], n_samples = class_4_value)
        classes[5] = resample(classes[5], n_samples = class_5_value)
        
        # Rejoin classes
        
        merged = pd.concat(classes)
        merged = merged.reset_index()
        merged = merged.drop('index', axis = 1)
        
        # X, y split again
        
        X_train_split = merged.drop(target_col, axis=1)
        y_train_split = merged[target_col]
            
        # XGB fit
        
        xgb_seed = random.randint(0, 2023)
        xgb = XGBClassifier(n_estimators = n_estimators,
                            learning_rate = learning_rate,
                            max_depth = max_depth,
                            subsample = subsample,
                            colsample_bytree = colsample_bytree,
                            random_state = xgb_seed).fit(X_train_split.values, y_train_split.values)
        
        # Keep track of feature importances for each XGB
        
        iter_feature_importance = iter_feature_importance + xgb.feature_importances_
        
        # XGB predict for valid data
        
        valid_prediction = xgb.predict(X_val.values)
        valid_prediction = pd.DataFrame(valid_prediction, index=X_val.index, columns = [f'XGB_Step_{step}_Fold_{i}']) 
        iter_valid_predictions.append(valid_prediction)
        
        # Store kappa (the metric for this data) results
        
        kappa_result = cohen_kappa_score(y_val, valid_prediction, weights='quadratic')
        kappas.append(kappa_result)
        
        # XGB predict for eval data
        
        eval_prediction = xgb.predict(eval_data[rand_cols].values)
        eval_prediction = pd.DataFrame(eval_prediction, index=eval_data.index, columns = [f'XGB_Step_{step}_Fold_{i}'])
        iter_eval_predictions.append(eval_prediction)
        
        # If we get a kappa of less than 0.495 on one the 5-fold slices then this doesn't look promissng and we 
        # skip to the next step in tuning iterations.
        
        if kappa_result < 0.495:
            kappas = [0]
            completed_iterations = completed_iterations - 1
            break
    
    valid_predictions.append(pd.concat(iter_valid_predictions, axis=1))
    eval_predictions.append(pd.concat(iter_eval_predictions, axis=1))
    feature_importances.append((1/cv_folds) * iter_feature_importance)
    
    tuning_results['kappa'].append(np.mean(kappas))
    tuning_results['n_estimators'].append(n_estimators)
    tuning_results['learning_rate'].append(learning_rate)
    tuning_results['max_depth'].append(max_depth)
    tuning_results['subsample'].append(subsample)
    tuning_results['colsample_bytree'].append(colsample_bytree)
    tuning_results['class_0_value'].append(class_0_value)
    tuning_results['class_1_value'].append(class_1_value)
    tuning_results['class_2_value'].append(class_2_value)
    tuning_results['class_3_value'].append(class_3_value)
    tuning_results['class_4_value'].append(class_4_value)
    tuning_results['class_5_value'].append(class_5_value)
    tuning_results[f'num_cols (of {X_train_origin.shape[1]})'].append(len(rand_cols))
    
    if len(iter_valid_predictions) == cv_folds:
        print(f'Step: {step} kappa: {np.mean(kappas)}')
    
tuning_results = pd.DataFrame(tuning_results)
tuning_results.sort_values(by = 'kappa', axis = 0, inplace = True, ascending = False)
tuning_results

Step: 36 kappa: 0.5284889955395059
Step: 68 kappa: 0.5371871035279868
Step: 116 kappa: 0.5358582275317664
Step: 180 kappa: 0.5261074050711608
Step: 230 kappa: 0.5193991577735966
Step: 255 kappa: 0.5304376700201586
Step: 302 kappa: 0.5228392238372865
Step: 345 kappa: 0.5375826019264174
Step: 410 kappa: 0.5281073120825818
Step: 508 kappa: 0.5179475556473625
Step: 527 kappa: 0.5149617730513935
Step: 652 kappa: 0.5239221639048848
Step: 672 kappa: 0.5162438341039733
Step: 708 kappa: 0.5202537150741168
Step: 796 kappa: 0.5287862408748393
Step: 808 kappa: 0.525304079374853
Step: 861 kappa: 0.5331271148734373
Step: 898 kappa: 0.51642832361235


,kappa,n_estimators,learning_rate,max_depth,subsample,colsample_bytree,class_0_value,class_1_value,class_2_value,class_3_value,class_4_value,class_5_value,num_cols (of 33)
345,0.537583,350,0.035556,2,0.25,0.25,10,75,418,418,369,24,20
68,0.537187,75,0.128889,2,0.50,0.75,32,53,654,418,421,61,19
116,0.535858,150,0.066667,2,0.75,0.50,53,75,713,536,306,24,18
861,0.533127,100,0.082222,3,0.75,0.25,17,32,595,300,432,24,27
255,0.530438,150,0.020000,2,0.25,0.50,61,10,536,418,264,39,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,0.000000,100,0.160000,3,0.90,0.50,24,46,300,477,348,39,26
343,0.000000,250,0.035556,5,0.75,0.25,46,61,359,713,421,75,27
344,0.000000,300,0.051111,5,0.50,0.25,39,32,713,772,274,32,11
346,0.000000,100,0.144444,2,0.25,0.90,46,39,477,418,400,61,14


In [6]:
tuning_results.head(10)

,kappa,n_estimators,learning_rate,max_depth,subsample,colsample_bytree,class_0_value,class_1_value,class_2_value,class_3_value,class_4_value,class_5_value,num_cols (of 33)
345,0.537583,350,0.035556,2,0.25,0.25,10,75,418,418,369,24,20
68,0.537187,75,0.128889,2,0.50,0.75,32,53,654,418,421,61,19
116,0.535858,150,0.066667,2,0.75,0.50,53,75,713,536,306,24,18
861,0.533127,100,0.082222,3,0.75,0.25,17,32,595,300,432,24,27
255,0.530438,150,0.020000,2,0.25,0.50,61,10,536,418,264,39,13
796,0.528786,150,0.020000,2,0.50,0.25,24,75,831,536,390,39,20
36,0.528489,250,0.051111,2,0.75,0.75,75,32,831,536,432,68,18
410,0.528107,250,0.020000,2,0.75,0.50,32,53,536,359,327,53,22
180,0.526107,50,0.051111,2,0.90,0.75,53,39,477,418,358,68,26
808,0.525304,200,0.051111,3,0.75,0.50,17,53,654,300,379,39,27


In [7]:
# Here we keep track of the feature importances for each XGB, not useful for the final model

#feature_frames = []

#for iter in range(50):
#    feature_frame = pd.DataFrame([cols_selection[iter], feature_importances[iter]])
#    feature_frame = feature_frame.transpose().sort_values(1, ascending=False).set_index(0)
#    feature_frames.append(feature_frame)

#pd.concat(feature_frames, axis=1).mean(axis=1).sort_values(ascending=False)

In [44]:
# We search for linear combinations of the best predictions. This is done by randomly selecting coefficients.
# This seems like a clumsy way of proceeding, so we should look for a better one.

# We check for combinations involving up to the 10 best predictions
checking_range = 10

# The best predictions. Here we need to sort_index, bizarely the Kaggle code didn't require this.
best_preds = [cv_folds * valid_predictions[tuning_results.index[i]].sort_index().mean(axis=1) for i in range(checking_range)]

# Initliatize and set parameters

best_score = 0
scaled_total = 0
splits = 40
linspc = np.linspace(0.0, 0.5, splits)

print(f'Searching over {linspc}')

# The loop

for num_preds in range(2,checking_range):
    print(f'Checking {num_preds}')
    total_probe = min(num_preds * 5000, splits**num_preds)
    for probe in range(total_probe):
        if probe % 5000 == 0:
            print(f'Check {probe}')
        rnd = random.choices(linspc, k=num_preds)
        scaled_total = np.matmul(np.array(rnd), np.array(best_preds[0:num_preds]))
        scaled_total = scaled_total.round()
        score = cohen_kappa_score(y_train, scaled_total, weights='quadratic')
        if score > best_score:
            print(scaled_total)
            best_score = score
            best_coefficients = rnd
            best_num_preds = num_preds
            print(f'New best score {score} with coefficients \n {rnd}')

Searching over [0.         0.01282051 0.02564103 0.03846154 0.05128205 0.06410256
 0.07692308 0.08974359 0.1025641  0.11538462 0.12820513 0.14102564
 0.15384615 0.16666667 0.17948718 0.19230769 0.20512821 0.21794872
 0.23076923 0.24358974 0.25641026 0.26923077 0.28205128 0.29487179
 0.30769231 0.32051282 0.33333333 0.34615385 0.35897436 0.37179487
 0.38461538 0.3974359  0.41025641 0.42307692 0.43589744 0.44871795
 0.46153846 0.47435897 0.48717949 0.5       ]
Checking 2
Check 0
[15. 14. 12. ... 15.  7. 15.]
New best score 0.04628579687968559 with coefficients 
 [0.2692307692307692, 0.47435897435897434]
[8. 8. 7. ... 8. 4. 8.]
New best score 0.1358214111434929 with coefficients 
 [0.10256410256410256, 0.3076923076923077]
[8. 7. 6. ... 8. 4. 8.]
New best score 0.14070902511727867 with coefficients 
 [0.14102564102564102, 0.24358974358974358]
[6. 6. 5. ... 6. 3. 6.]
New best score 0.2590801139525786 with coefficients 
 [0.11538461538461538, 0.1923076923076923]
[6. 5. 5. ... 6. 3. 6.]
New b

# Results and Submission

In [45]:
result = [eval_predictions[tuning_results.index[i]].fillna(0).mean(axis=1) for i in range(best_num_preds)]

result = np.array(result)

result = pd.DataFrame(np.matmul(best_coefficients, result).round()).astype('int64')

submission = sample_submit.copy()

submission[target_col] = result
submission[target_col] = le.inverse_transform(submission[target_col])

submission.to_csv('submission.csv', index=False)